In [101]:
import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_columns = None
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [102]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Assuming df is your DataFrame and 'Feature17' has missing values
imputer = IterativeImputer(max_iter=100, tol=1e-3)

df_train = pd.read_csv('../data/iith_foml_2023_train.csv')
# Fit the imputer model on the dataset to learn the data patterns
# df_train['Feature 16'].fillna(df_train['Feature 16'].mean(), inplace=True)
# df_train['Feature 17'].fillna(df_train['Feature 17'].mean(), inplace=True)
df_train.drop(columns=['Feature 16', 'Feature 17'], inplace=True)
imputer.fit(df_train)

# Transform the dataset to replace missing values
df_imputed = imputer.transform(df_train)

# Convert back to a DataFrame
df_imputed = pd.DataFrame(df_imputed, columns=df_train.columns)

In [103]:
X_train = df_imputed.iloc[:, :-1]
y_train = df_imputed.iloc[:, -1]
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [104]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Shape of train_x:", train_x.shape)
print("Shape of val_x:", val_x.shape)
print("Shape of train_y:", train_y.shape)
print("Shape of val_y:", val_y.shape)


Shape of train_x: (795, 22)
Shape of val_x: (199, 22)
Shape of train_y: (795,)
Shape of val_y: (199,)


In [105]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Initialize Adaboost model
model = BaggingClassifier(random_state=42)

# Train the model on the training data
model.fit(train_x, train_y)

# Make predictions on the validation data
predictions = model.predict(val_x)

# Calculate accuracy and macro F1 score
accuracy = accuracy_score(val_y, predictions)
macro_f1 = f1_score(val_y, predictions, average='macro', zero_division=0.0)

# Display results
print("Bagging Accuracy:", accuracy)
print("Bagging Macro F1 Score:", macro_f1)

# Classification report for more details
classification_rep = classification_report(val_y, predictions, zero_division=0.0)
print("\nBagging Classification Report:\n", classification_rep)

Bagging Accuracy: 0.864321608040201
Bagging Macro F1 Score: 0.46113991011643485

Bagging Classification Report:
               precision    recall  f1-score   support

         0.0       0.89      0.94      0.91        50
         1.0       0.93      0.98      0.95        91
         2.0       0.65      0.77      0.70        26
         3.0       0.00      0.00      0.00         1
         4.0       1.00      1.00      1.00         1
         5.0       0.50      0.38      0.43         8
         6.0       1.00      1.00      1.00        11
         7.0       0.00      0.00      0.00         2
         8.0       0.00      0.00      0.00         5
        11.0       0.00      0.00      0.00         1
        13.0       0.00      0.00      0.00         1
        14.0       1.00      1.00      1.00         1
        15.0       0.00      0.00      0.00         1

    accuracy                           0.86       199
   macro avg       0.46      0.47      0.46       199
weighted avg       0.